### TODO List
1. Move the data loading functions to another file that we can import
1. The data is sorted, needs to be shuffled

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import gzip
import simplejson
import json

In [2]:
def parse(filename):
    f = gzip.open(filename, 'r')
    entry = {}
    for l in f:
        l = str(l.strip())
        colonPos = l.find(':')
        if colonPos == -1:
            yield entry
            entry = {}
            continue
        eName = l[:colonPos][2:]
        rest = l[colonPos + 2:-1]
        entry[eName] = rest
    yield entry


def parse2(path):
    g = gzip.open(path, 'rt')
    for l in g:
        yield json.loads(l)

In [25]:
class WideAndDeep(object):
    def __init__(self, path, test_fraction):
        data = [p for p in parse2(path)]
        train_fraction = 1 - test_fraction
        self.num_reviews = len(data)
        
        # thank god this works... i did NOT feel like converting json to a dataframe...
        df = pd.DataFrame(data)
        
        # convert user ids to integers
        self.reviewer_ids = df["reviewerID"].unique()
        print(self.reviewer_ids[:10])
        return
        
        # convert product ids to integers
        self.asin = df["asin"].unique()
        
        # user-rating matrix.
        self.user_score = np.zeros((self.num_user_ids, self.num_product_ids))
        for idx, row in df.iterrows():
            self.user_score[idx, self.product_ids_to_numbers[row["asin"]]] = row["overall"]
        
        # this might be useful later when we implement the text-analysis version
        self.user_text = {}
        self.product_text = {}
        for idx, review in df.iterrows():
            text = review[keys["text"]]
            try:
                self.user_text[review[keys["userId"]]].append(text)
            except:
                self.user_text[review[keys["userId"]]] = [text]
            try:
                self.product_text[review[keys["productId"]]].append(text)
            except:
                self.product_text[review[keys["productId"]]] = [text]
    
    def define_feature_columns():
        helpfulness = tf.feature_column.numeric_column("review/helpfulness")
        user_id = tf.feature_column.categorical_column_with_identity("review/userId",
                                                                     num_buckets=self.num_user_ids)
        product_id = tf.feature_column.categorical_column_with_identity("product/productId",
                                                                        num_buckets=self.num_product_ids)
        profile_name = tf.feature_column.categorical_column_with_identity("review/profileName",
                                                                         num_buckets=self.num_profile_names)

In [26]:
wad = WideAndDeep("data/reviews_Amazon_Instant_Video_5.json.gz", 0.1)

# g = gzip.open("data/reviews_Amazon_Instant_Video_5.json.gz", 'rt')
# next(g)
# json.loads(str(next(g)))

['A11N155CW1UV02' 'A3BC8O2KCL29V2' 'A60D5HQFOTSOM' 'A1RJPIGRSNX4PW'
 'A16XRPF40679KG' 'A1POFVVXUZR3IQ' 'A1PG2VV4W1WRPL' 'ATASGS8HZHGIB'
 'A3RXD7Z44T9DHW' 'AUX8EUBNTHIIU']


In [27]:
# col 0 is user ids, col 1 is product ids
pd.read_csv("data/ratings_Amazon_Instant_Video.csv", header=None)

,0,1,2,3
0,A1EE2E3N7PW666,B000GFDAUG,5.0,1202256000
1,AGZ8SM1BGK3CK,B000GFDAUG,5.0,1198195200
2,A2VHZ21245KBT7,B000GIOPK2,4.0,1215388800
3,ACX8YW2D5EGP6,B000GIOPK2,4.0,1185840000
4,A9RNMO9MUSMTJ,B000GIOPK2,2.0,1281052800
5,A3STFVPM8NHJ7B,B000GIOPK2,5.0,1203897600
6,A2582KMXLK2P06,B000GIOPK2,5.0,1205884800
7,A1TZCLCW9QGGBH,B000GIOPK2,4.0,1209427200
8,A2E2I6B878CRMA,B000GIOPK2,5.0,1378684800
9,AD5MZA8SOVMPJ,B000GIOPK2,5.0,1218240000
